In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import re

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 71.9 MB/s eta 0:00:00


In [ ]:
import gensim
from gensim import corpora, models, similarities

import nltk
from nltk.corpus import stopwords

In [ ]:
import scipy
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd

In [ ]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv(r'0_news_articles.csv')
data.head()

,Article_id,Title,Description,Date,Category,URL
0,0,Fire at Vaishno Devi shrine complex; cash coun...,"No one was injured in the fire, which broke ou...","June 8, 2021 7:28:32 pm",India,https://indianexpress.com/article/india/vaishn...
1,1,"Had not gone to meet Nawaz Sharif, says Uddhav...",Uddhav Thackeray led a delegation of his cabin...,"June 8, 2021 6:56:40 pm",India,https://indianexpress.com/article/india/had-no...
2,2,Corruption case: Former Haryana I-T deputy com...,It was in 2016 that the CBI had arrested Nitin...,"June 8, 2021 6:25:24 pm",India,https://indianexpress.com/article/india/corrup...
3,3,Kannur MP K Sudhakaran appointed chief of Cong...,Sudhakaran will replace Ramachandran who had a...,"June 8, 2021 5:04:40 pm",India,https://indianexpress.com/article/india/sudhak...
4,4,"Kerala girl of Class 5 writes to CJI, lauds SC...",Chief Justice N V Ramana responded to the Clas...,"June 8, 2021 4:43:10 pm",India,https://indianexpress.com/article/india/kerala...


In [ ]:
rating = pd.read_csv(r'3_user_rated_articles.csv')
print(rating.shape)
rating.drop(columns= rating.columns[0],
        axis=1,
        inplace=True)
rating.head()

(1113, 8)


,Article_id,Title,UserId,SessionId,Article Rank,Time Spent (seconds),Ratings
0,0,Fire at Vaishno Devi shrine complex; cash coun...,1,1,1,25,2
1,1,"Had not gone to meet Nawaz Sharif, says Uddhav...",1,1,2,14,4
2,2,Corruption case: Former Haryana I-T deputy com...,1,1,3,57,1
3,4,"Kerala girl of Class 5 writes to CJI, lauds SC...",1,1,5,62,2
4,8,Dantewada: 24-year-old tribal woman killed in ...,1,1,9,77,3


In [ ]:
rating.tail()

,Article_id,Title,UserId,SessionId,Article Rank,Time Spent (seconds),Ratings
1108,2244,How learning about your genes can help with yo...,2234,2229,5,28,4
1109,2245,"Malaika Arora shares yoga asanas for healthy, ...",2235,2230,6,89,2
1110,2246,COVID-19 diet: Khichdi is a ‘safe’ option; oth...,2236,2231,7,46,4
1111,2247,‘Keep listening. The world wants to hear your ...,2237,2232,8,47,4
1112,2248,"Forget cold drinks, switch to refreshing bael ...",2238,2233,9,34,3


In [ ]:
n_users = int(rating.UserId.nunique())
n_article = int(rating.Article_id.nunique())
print("Number of users: " , n_users)
print("Number of articles: ", n_article)

Number of users:  1107
Number of articles:  1113


In [ ]:
user_pivot = rating.pivot_table(index = 'UserId', columns = 'Article_id', values = 'Ratings')
user_pivot.head()

Article_id,0,1,2,4,8,9,11,12,13,14,...,2236,2237,2238,2241,2242,2244,2245,2246,2247,2248
UserId,,,,,,,,,,,,,,,,,,,,,
1,2.0,4.0,1.0,2.0,3.0,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_pivot.shape

(1107, 1113)

In [ ]:
user_pivot = user_pivot.fillna(0)
user_pivot_matrix = user_pivot.values
user_pivot_matrix

array([[2., 4., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 4., 0., 0.],
       [0., 0., 0., ..., 0., 4., 0.],
       [0., 0., 0., ..., 0., 0., 3.]])

In [ ]:
user_item_pivot_sparse = csr_matrix(user_pivot)
user_item_pivot_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1113 stored elements and shape (1107, 1113)>

In [ ]:
n_factors = 150
U, sigma, Vt = svds(user_pivot_matrix, k = n_factors)

sigma = np.diag(sigma)
sigma.shape

(150, 150)

In [ ]:
Vt

array([[-9.49620428e-18, -6.20622873e-17, -2.45866829e-17, ...,
         5.33588119e-19,  8.16902077e-18,  2.79908400e-19],
       [ 3.02857065e-18,  3.39042519e-17, -5.37613553e-19, ...,
        -4.30971261e-18,  8.44663921e-18, -1.31221025e-19],
       [ 1.41859341e-17,  5.23806035e-17,  1.03794287e-18, ...,
         1.49908983e-18, -1.11747774e-17, -1.64418647e-20],
       ...,
       [ 3.20820233e-17,  4.47907365e-17, -2.27862956e-17, ...,
        -1.68636855e-18,  3.28291619e-17,  1.79656157e-20],
       [ 5.88417872e-17,  1.18836542e-16, -6.71480090e-18, ...,
         7.10158403e-18,  1.41482157e-17, -7.93405325e-19],
       [-2.30940108e-01, -4.61880215e-01, -1.15470054e-01, ...,
         4.68159654e-33,  4.00617226e-19, -3.23421608e-20]])

In [ ]:
all_user_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_ratings_norm = (all_user_ratings - all_user_ratings.min()) / (all_user_ratings.max() - all_user_ratings.min())
all_user_ratings_norm

array([[0.47314843, 0.82438281, 0.29753124, ..., 0.12191405, 0.12191405,
        0.12191405],
       [0.12191405, 0.12191405, 0.12191405, ..., 0.12191405, 0.12191405,
        0.12191405],
       [0.12191405, 0.12191405, 0.12191405, ..., 0.12191405, 0.12191405,
        0.12191405],
       ...,
       [0.12191405, 0.12191405, 0.12191405, ..., 0.12191405, 0.12191405,
        0.12191405],
       [0.12191405, 0.12191405, 0.12191405, ..., 0.12191405, 0.12191405,
        0.12191405],
       [0.12191405, 0.12191405, 0.12191405, ..., 0.12191405, 0.12191405,
        0.12191405]])

In [ ]:
cf_preds_df = pd.DataFrame(all_user_ratings_norm, columns = user_pivot.columns).transpose()

In [ ]:
class Collaborative:

    name = "Collaborative Filter"

    def __init__(self, predictions, items = None):
        self.predictions = predictions
        self.items = items

    def get_model_name(self):
        return self.name

    def recommend_items(self, user_id, items_ignore = [], topn = 10, verbose = False):
        sorted_preds = self.predictions[user_id].sort_values(ascending = False).reset_index()

        recommendations = sorted_preds[~sorted_preds['Article_id'].isin(items_ignore)].head(topn)

        if verbose:
            if self.items is None:
                raise Exception('"items" required in verbose mode')

            recommendations = recommendations.merge(self.items, how = 'left', left_on = 'Article_id',
                                                    right_on = 'Article_id')[['Article_id', 'Title']]


        return recommendations

In [ ]:
model = Collaborative(cf_preds_df, data)

In [ ]:
model.recommend_items(user_id = 224, verbose = True)

,Article_id,Title
0,1615,"‘If women are given a voice, they use it; when..."
1,1491,Indian women’s hockey team can create history ...
2,16,‘Relations with China at a crossroads’: S Jais...
3,1513,‘I’m not just one colour…I’m the rainbow’: Shi...
4,1720,‘Eating even when I was full was the culprit’:...
5,1122,ECB not changing India Test series schedule to...
6,1251,"Max Verstappen wins Monaco GP, takes F1 title ..."
7,1703,Bhumi Pednekar to Rakul Preet Singh: Four acto...
8,431,"PM announces aid, free education for Covid orp..."
9,264,Preparation for third covid wave: Panchkula ad...


In [ ]:
user_pivot.head()

Article_id,0,1,2,4,8,9,11,12,13,14,...,2236,2237,2238,2241,2242,2244,2245,2246,2247,2248
UserId,,,,,,,,,,,,,,,,,,,,,
1,2.0,4.0,1.0,2.0,3.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
u_id = list(user_pivot.index)
u_dict = {}
counter = 0

for i in u_id:
    u_dict[i] = counter
    counter += 1
len(u_dict)

1107

In [ ]:
# convert to csr matrix
u_interaction_csr = csr_matrix(user_pivot.values)
u_interaction_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1113 stored elements and shape (1107, 1113)>

In [ ]:
item_dict ={}
df = rating[['Article_id', 'Title']].sort_values('Article_id').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'Article_id'])] = df.loc[i,'Title']

In [ ]:
len(item_dict)

1113

In [ ]:
title = list(data['Title'])
title[:10]

['Fire at Vaishno Devi shrine complex; cash counter damaged',
 'Had not gone to meet Nawaz Sharif, says Uddhav Thackeray as he plays down one-on-one meeting with PM Modi',
 'Corruption case: Former Haryana I-T deputy commissioner gets 4 years in prison',
 'Kannur MP K Sudhakaran appointed chief of Congress in Kerala',
 'Kerala girl of Class 5 writes to CJI, lauds SC for saving lives in fight with Covid',
 'Madhya Pradesh govt gets HC notice on communal clashes during fundraising for Ram temple',
 'Uddhav Thackeray meets PM Modi; discusses Maratha quota issue, GST compensation',
 'New Covid-19 vaccination guidelines out, allocation based on state population',
 'Dantewada: 24-year-old tribal woman killed in ‘maoist encounter’; family claims it was staged, alleges rape',
 'Fire at TMC MLA Madan Mitra’s residence in Kolkata']

In [ ]:
total = data.isnull().sum().sort_values(ascending = False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head()

,Total,Percent
Article_id,0,0.0
Title,0,0.0
Description,0,0.0
Date,0,0.0
Category,0,0.0
